In [22]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
from datetime import datetime
import pytz

In [25]:
sw_lat, sw_long = 38.757, -94.908
ne_lat, ne_long = 39.427, -94.235
query = f"""
[out:json];
(
  node["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  way["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  relation["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
);
out center;
"""

response = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
data = response.json()
for el in data["elements"]:
  name = el.get("tags", {}).get("name")
  lat = el.get("lat")
  lon = el.get("lon")
  if lat == None:
    lat = el.get("center", {}).get("lat")
  if lon == None:
    lon = el.get("center", {}).get("lon")
  if name != None:
    print(f'{name}: {lat}, {lon}')
  # print(el)

Posse Course: 39.4213658, -94.5476547
The Hill Course: 38.986333, -94.5238809
The Rock Course: 38.9893881, -94.5222873
The River Course: 38.9919431, -94.5278009
Paradise Pointe: 39.4232906, -94.5514754
Brough Creek National GC: 39.1671392, -94.7610853
Smiley's Golf Complex: 38.9440764, -94.8480508
Overland Park Golf Course: 38.90372, -94.7189616
Westlinks Golf Course: 38.8926212, -94.7335267
Swope Memorial Golf Course: 38.9988071, -94.5131265
Heart of America Golf Course: 38.991176, -94.5260483
St Andrews Golf Club: 38.8787932, -94.7118029
Lake Quivira Country Club: 39.0455078, -94.7657634
Milburn Country Club: 39.0040805, -94.6767136
Tomahawk Hills Golf Course: 38.9971895, -94.7900334
Royal Meadows Golf Club: 39.0336354, -94.4583346
The Kansas City Country Club: 39.0215514, -94.6208824
Sycamore Ridge Golf Club: 38.7565789, -94.840674
Wolf Creek Golf: 38.7927887, -94.7537362
Driving Range: 39.2654783, -94.5128536
Hodge Park Golf Course: 39.2532934, -94.4939931
Shoal Creek Golf Course: 

In [23]:
class GolfCourse:
  def __init__(self, id, name, lat, lon, source):
    self.id = id
    self.name = name
    self.lat = lat
    self.lon = lon
    self.source = source

  def to_dict(self):
    return {
      "id": self.id,
      "name": self.name,
      "lat": self.lat,
      "lon": self.lon,
      "source": self.source
    }

  def __repr__(self):
    return f"GolfCourse(name={self.name}, lat={self.lat}, lon={self.lon}), source={self.source})"

In [8]:
# Coordinates you posted — cleaned into a dictionary
coordinates = {
  'Shoal Creek Golf Course': (39.2528261, -94.4788154),
  'Hodge Park Golf Course': (39.2532934, -94.4939931),
  'Winterstone Golf Course': (39.1250818, -94.3787228),
  'Adams Pointe Golf Course': (39.019879, -94.2414334),
  'Heart of America Golf Course': (38.991176, -94.5260483),
  'Sycamore Ridge Golf Club': (38.7565789, -94.840674),
  'Paradise Pointe Golf Course - The Outlaw': (39.4232906, -94.5514754),  # Using Paradise Pointe
  'Paradise Pointe Golf Course - The Posse': (39.4230916, -94.5472098),  # Posse
  'Falcon Lakes Golf Club': (39.1650505, -94.910161),
  "Dub's Dread Golf Club": (39.1685497, -94.8752899),
  'Painted Hills Golf Club': (39.1242683, -94.7417212),
  'Drumm Farm Golf Club - Full': (39.0620829, -94.3981964),
  'Drumm Farm Golf Club - Executive': (39.0620829, -94.3981964),  # Same coords
  'Royal Meadows Golf Club': (39.0336354, -94.4583346),
  'Teetering Rocks Golf Course': (38.9628262, -94.4396519),
  'Heritage Park Golf Course': (38.8273693, -94.7550834),
  'Tomahawk Hills Golf Course': (38.9971895, -94.7900334),
  'Sunflower Hills Golf Course': (39.1037935, -94.8676643),
}

In [9]:
bookateetime_courses = {
  'Shoal Creek Golf Course': '118-1',
  'Hodge Park Golf Course': '117-1',
  'Winterstone Golf Course': '62-1',
  'Adams Pointe Golf Course': '45-1',
  'Heart of America Golf Course': '49-1',
  'Sycamore Ridge Golf Club': '44-1',
  'Paradise Pointe Golf Course - The Outlaw': '24-1',
  'Paradise Pointe Golf Course - The Posse': '24-2',
}

chronogolf_courses = {
  'Falcon Lakes Golf Club': 6633,
}

golfback_courses = {
  "Dub's Dread Golf Club": '398d44ce-a908-4ce7-8f50-e5f4bdc77b73',
  'Painted Hills Golf Club': '857a12d4-a9cf-4a43-afe2-60940bdc7438',
  'Drumm Farm Golf Club - Full': 'd70999c9-d7d4-4008-9f41-4e9551b3c796',
  'Drumm Farm Golf Club - Executive': '9a1de435-8a46-4840-9cdc-332c3cfea782',
  'Royal Meadows Golf Club': 'd2278228-4700-4354-95a8-422a8f9a5a16'
}

foreup_courses = {
  'Teetering Rocks Golf Course': 7341,
  'Heritage Park Golf Course': 12159,
  'Tomahawk Hills Golf Course': 11026,
}

loners = {
  'Sunflower Hills Golf Course': 'https://www.sunflowerhillsgolfcourse.com/TeeTimes',
}

In [15]:
courses = []

# Helper to add courses
def add_courses(source_name, courses_dict):
  for name, id in courses_dict.items():
    coord = coordinates.get(name)
    if coord:
      lat, lon = coord
      courses.append(GolfCourse(id=id, name=name, lat=lat, lon=lon, source=source_name))
    else:
      print(f"[Warning] No coordinates found for: {name}")

add_courses('bookateetime', bookateetime_courses)
add_courses('chronogolf', chronogolf_courses)
add_courses('golfback', golfback_courses)
add_courses('foreup', foreup_courses)
add_courses('loner', loners)

In [16]:
for course in courses:
  print(course)

GolfCourse(name=Shoal Creek Golf Course, lat=39.2528261, lon=-94.4788154), source=bookateetime)
GolfCourse(name=Hodge Park Golf Course, lat=39.2532934, lon=-94.4939931), source=bookateetime)
GolfCourse(name=Winterstone Golf Course, lat=39.1250818, lon=-94.3787228), source=bookateetime)
GolfCourse(name=Adams Pointe Golf Course, lat=39.019879, lon=-94.2414334), source=bookateetime)
GolfCourse(name=Heart of America Golf Course, lat=38.991176, lon=-94.5260483), source=bookateetime)
GolfCourse(name=Sycamore Ridge Golf Club, lat=38.7565789, lon=-94.840674), source=bookateetime)
GolfCourse(name=Paradise Pointe Golf Course - The Outlaw, lat=39.4232906, lon=-94.5514754), source=bookateetime)
GolfCourse(name=Paradise Pointe Golf Course - The Posse, lat=39.4230916, lon=-94.5472098), source=bookateetime)
GolfCourse(name=Falcon Lakes Golf Club, lat=39.1650505, lon=-94.910161), source=chronogolf)
GolfCourse(name=Dub's Dread Golf Club, lat=39.1685497, lon=-94.8752899), source=golfback)
GolfCourse(nam

In [29]:
courses[-3].id

12159

In [18]:
import json

# Convert to list of dicts
courses_json = [course.to_dict() for course in courses]

# Option 1: Write to local file
with open('../golf_courses.json', 'w') as f:
    json.dump(courses_json, f, indent=2)

print("Saved to golf_courses.json ✅")

Saved to golf_courses.json ✅


In [24]:
with open('../golf_courses.json', 'r') as f:
  courses_data = json.load(f)

courses = [GolfCourse(**course) for course in courses_data]

In [ ]:
date = '2025-05-02'
players = 4

tee_times_df = pd.DataFrame()

for course in courses:
  if course.source == 'bookateetime':
    # Make a GET request to the URL with the course name and date
    response = requests.get(f"https://bookateetime.teequest.com/search/{course.id}/{date}?selectedPlayers={players}&selectedHoles=18")

    # Decode the response content as a string
    content_str = response.content.decode('utf-8')
    soup = BeautifulSoup(content_str, 'html.parser')
    tee_times = []

    for tee_time_div in soup.find_all('div', class_='tee-time'):
      href = tee_time_div.find('a', class_='btn')['href']
      tt = {
        'course': course.name,
        'tee_time': pd.to_datetime(tee_time_div['data-date-time'], format='%Y%m%d%H%M'), #tee_time_div['data-date-time'],
        'price': float(tee_time_div['data-price']),
        'players': int(tee_time_div['data-available']),
        'lat': course.lat,
        'lon': course.lon,
        'book_url': f'https://bookateetime.teequest.com{href}'
      }
      tee_times.append(tt)

    tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)

  if course.source == 'golfback':
    url = f"https://api.golfback.com/api/v1/courses/{course.id}/date/{date}/teetimes"
    headers = {
      "User-Agent": "Mozilla/5.0",
      "Referer": "https://golfback.com/",
      "Content-Type": "application/json",
    }
    params = {
      "date": date,
      "course_id": course.id,
      "players": players
    }

    response = requests.post(url, headers=headers, json=params)
    tee_times_raw = response.json()['data']
    
    tee_times = []
    for tee_time in tee_times_raw:
      tt = {
        'course': course.name,
        'tee_time': pd.to_datetime(tee_time['dateTime'], format='%Y-%m-%dT%H:%M:%S%z') \
          .astimezone(pytz.timezone("US/Central")).strftime("%Y-%m-%d %H:%M:%S"),
        'price': float(tee_time['rates'][0]['price']),
        'players': tee_time['playersMax'],
        'lat': course.lat,
        'lon': course.lon,
        'book_url': f'https://golfback.com/#/course/{course.id}/date/{date}/teetime/{tee_time['id']}?rateId={tee_time['rates'][0]['ratePlanId']}&holes=18&players={players}'
      }
      tee_times.append(tt)

    tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)
  
  if course.source == 'foreup':
    flip_date = datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y')
    url = f"https://foreupsoftware.com/index.php/api/booking/times?time=all&date={flip_date}&holes=all&players={players}&booking_class=14824&schedule_id={course.id}&api_key=no_limits"
    headers = {
      "User-Agent": "Mozilla/5.0",
      "Referer": f"https://foreupsoftware.com/index.php/booking/22857/7340",
      "Content-Type": "application/json",
    }
    params = {
      "date": datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y'),
      "players": players,
    }

    response = requests.get(url, headers=headers, json=params)
    tee_times_raw = response.json()

    tee_times = []
    for tee_time in tee_times_raw:
      # print(tee_time)
      # break
      tt = {
        'course': course.name,
        'tee_time': pd.to_datetime(tee_time['time'], format='%Y-%m-%d %H:%M'),
        'price': float(tee_time['green_fee'] + tee_time['cart_fee']),
        'players': tee_time['available_spots'],
        'lat': course.lat,
        'lon': course.lon,
        'book_url': f'https://foreupsoftware.com/index.php/booking/22857/{course.id}#/teetimes'
      }
      tee_times.append(tt)

    tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)

In [27]:
tee_times_df

,course,tee_time,price,players,lat,lon,book_url
0,Shoal Creek Golf Course,2025-05-02 16:20:00,50.0,4,39.252826,-94.478815,https://bookateetime.teequest.com/teetime/118-...
1,Shoal Creek Golf Course,2025-05-02 16:30:00,50.0,4,39.252826,-94.478815,https://bookateetime.teequest.com/teetime/118-...
2,Shoal Creek Golf Course,2025-05-02 16:40:00,50.0,4,39.252826,-94.478815,https://bookateetime.teequest.com/teetime/118-...
3,Shoal Creek Golf Course,2025-05-02 16:50:00,50.0,4,39.252826,-94.478815,https://bookateetime.teequest.com/teetime/118-...
4,Shoal Creek Golf Course,2025-05-02 17:00:00,50.0,4,39.252826,-94.478815,https://bookateetime.teequest.com/teetime/118-...
...,...,...,...,...,...,...,...
430,Heritage Park Golf Course,2025-05-02 07:30:00,48.0,4,38.827369,-94.755083,NaN
431,Tomahawk Hills Golf Course,2025-05-02 16:10:00,31.0,4,38.997189,-94.790033,NaN
432,Tomahawk Hills Golf Course,2025-05-02 16:20:00,31.0,4,38.997189,-94.790033,NaN
433,Tomahawk Hills Golf Course,2025-05-02 16:40:00,31.0,4,38.997189,-94.790033,NaN


In [ ]:
date = '2025-05-02'
players = 4

tee_times_df = pd.DataFrame()

for course in bookateetime_courses.items():
  # Make a GET request to the URL with the course name and date
  response = requests.get(f"https://bookateetime.teequest.com/search/{course[1]}/{date}?selectedPlayers={players}&selectedHoles=18")

  # Decode the response content as a string
  content_str = response.content.decode('utf-8')
  soup = BeautifulSoup(content_str, 'html.parser')
  tee_times = []

  for tee_time_div in soup.find_all('div', class_='tee-time'):
    href = tee_time_div.find('a', class_='btn')['href']
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time_div['data-date-time'], format='%Y%m%d%H%M'), #tee_time_div['data-date-time'],
      'price': float(tee_time_div['data-price']),
      'players': int(tee_time_div['data-available']),
      'book_url': f'https://bookateetime.teequest.com{href}'
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)

# Convert the 'date_time' column to datetime format
# tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], format='%Y%m%d%H%M')

In [6]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show full column content

tee_times_df[
  tee_times_df['course'] == 'Paradise Pointe Golf Course - The Posse'
].sort_values('tee_time').head(10)

,course,tee_time,price,players,book_url
105,Paradise Pointe Golf Course - The Posse,2025-05-02 08:30:00,46.0,4,https://bookateetime.teequest.com/teetime/24-2/202505020830/1/A/CS/4/18
106,Paradise Pointe Golf Course - The Posse,2025-05-02 12:50:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021250/1/A/AO/4/18
107,Paradise Pointe Golf Course - The Posse,2025-05-02 13:20:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021320/1/A/AO/4/18
108,Paradise Pointe Golf Course - The Posse,2025-05-02 13:40:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021340/1/A/AO/4/18
109,Paradise Pointe Golf Course - The Posse,2025-05-02 13:50:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021350/1/A/AO/4/18
110,Paradise Pointe Golf Course - The Posse,2025-05-02 14:00:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021400/1/A/AO/4/18
111,Paradise Pointe Golf Course - The Posse,2025-05-02 14:10:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021410/1/A/AO/4/18
112,Paradise Pointe Golf Course - The Posse,2025-05-02 14:20:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021420/1/A/AO/4/18
113,Paradise Pointe Golf Course - The Posse,2025-05-02 14:30:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021430/1/A/AO/4/18
114,Paradise Pointe Golf Course - The Posse,2025-05-02 14:40:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202505021440/1/A/AO/4/18


In [7]:
for course in golfback_courses.items():
  url = f"https://api.golfback.com/api/v1/courses/{course[1]}/date/{date}/teetimes"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://golfback.com/",
    "Content-Type": "application/json",
  }
  params = {
    "date": date,
    "course_id": course[1],
    "players": players
  }

  response = requests.post(url, headers=headers, json=params)
  tee_times_raw = response.json()['data']
  
  tee_times = []
  for tee_time in tee_times_raw:
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['dateTime'], format='%Y-%m-%dT%H:%M:%S%z') \
        .astimezone(pytz.timezone("US/Central")).strftime("%Y-%m-%d %H:%M:%S"),
      'price': float(tee_time['rates'][0]['price']),
      'players': tee_time['playersMax'],
      'book_url': f'https://golfback.com/#/course/{course[1]}/date/{date}/teetime/{tee_time['id']}?rateId={tee_time['rates'][0]['ratePlanId']}&holes=18&players={players}'
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


In [8]:
for course in foreup_courses.items():
  flip_date = datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y')
  url = f"https://foreupsoftware.com/index.php/api/booking/times?time=all&date={flip_date}&holes=all&players={players}&booking_class=14824&schedule_id={course[1]}&api_key=no_limits"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": f"https://foreupsoftware.com/index.php/booking/22857/7340",
    "Content-Type": "application/json",
  }
  params = {
    "date": datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y'),
    "players": players,
  }

  response = requests.get(url, headers=headers, json=params)
  tee_times_raw = response.json()

  tee_times = []
  for tee_time in tee_times_raw:
    # print(tee_time)
    # break
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['time'], format='%Y-%m-%d %H:%M'),
      'price': float(tee_time['green_fee'] + tee_time['cart_fee']),
      'players': tee_time['available_spots']
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


In [9]:
tee_times_df.head()

,course,tee_time,price,players,book_url
0,Shoal Creek Golf Course,2025-05-02 16:20:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021620/1/A/CS/4/18
1,Shoal Creek Golf Course,2025-05-02 16:30:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021630/1/A/CS/4/18
2,Shoal Creek Golf Course,2025-05-02 16:40:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021640/1/A/CS/4/18
3,Shoal Creek Golf Course,2025-05-02 16:50:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021650/1/A/CS/4/18
4,Shoal Creek Golf Course,2025-05-02 17:00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021700/1/A/CS/4/18


In [10]:
tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], utc=True)

In [13]:
tee_times_df.head()

,course,tee_time,price,players,book_url
0,Shoal Creek Golf Course,2025-05-02 16:20:00+00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021620/1/A/CS/4/18
1,Shoal Creek Golf Course,2025-05-02 16:30:00+00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021630/1/A/CS/4/18
2,Shoal Creek Golf Course,2025-05-02 16:40:00+00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021640/1/A/CS/4/18
3,Shoal Creek Golf Course,2025-05-02 16:50:00+00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021650/1/A/CS/4/18
4,Shoal Creek Golf Course,2025-05-02 17:00:00+00:00,50.0,4,https://bookateetime.teequest.com/teetime/118-1/202505021700/1/A/CS/4/18


In [14]:
tee_times_df[
  (tee_times_df['tee_time'].dt.hour >= 16) &
  (tee_times_df['price'] <= 90)
].sort_values(by=['course', 'tee_time']).head()#.to_csv('tee_times.csv', index=False)

,course,tee_time,price,players,book_url
68,Adams Pointe Golf Course,2025-05-02 16:09:00+00:00,45.0,4,https://bookateetime.teequest.com/teetime/45-1/202505021609/1/A/AO/4/18
69,Adams Pointe Golf Course,2025-05-02 16:18:00+00:00,45.0,4,https://bookateetime.teequest.com/teetime/45-1/202505021618/1/A/AO/4/18
70,Adams Pointe Golf Course,2025-05-02 16:27:00+00:00,45.0,4,https://bookateetime.teequest.com/teetime/45-1/202505021627/1/A/AO/4/18
71,Adams Pointe Golf Course,2025-05-02 16:36:00+00:00,45.0,4,https://bookateetime.teequest.com/teetime/45-1/202505021636/1/A/AO/4/18
72,Adams Pointe Golf Course,2025-05-02 16:45:00+00:00,45.0,4,https://bookateetime.teequest.com/teetime/45-1/202505021645/1/A/AO/4/18


In [12]:
tee_times_df.sort_values(by=['course', 'tee_time']).to_csv('tee_times.csv', index=False)

In [30]:
response = requests.get('https://tee-time-service-1091750267004.us-central1.run.app/tee_times?date=2025-05-03&players=4')

In [31]:
response.json()

[{'book_url': 'https://bookateetime.teequest.com/teetime/118-1/202505031620/1/A/CS/4/18',
  'course': 'Shoal Creek Golf Course',
  'lat': 39.2528261,
  'lon': -94.4788154,
  'players': 4,
  'price': 50.0,
  'tee_time': 'Sat, 03 May 2025 16:20:00 GMT'},
 {'book_url': 'https://bookateetime.teequest.com/teetime/118-1/202505031630/1/A/CS/4/18',
  'course': 'Shoal Creek Golf Course',
  'lat': 39.2528261,
  'lon': -94.4788154,
  'players': 4,
  'price': 50.0,
  'tee_time': 'Sat, 03 May 2025 16:30:00 GMT'},
 {'book_url': 'https://bookateetime.teequest.com/teetime/118-1/202505031640/1/A/CS/4/18',
  'course': 'Shoal Creek Golf Course',
  'lat': 39.2528261,
  'lon': -94.4788154,
  'players': 4,
  'price': 50.0,
  'tee_time': 'Sat, 03 May 2025 16:40:00 GMT'},
 {'book_url': 'https://bookateetime.teequest.com/teetime/118-1/202505031650/1/A/CS/4/18',
  'course': 'Shoal Creek Golf Course',
  'lat': 39.2528261,
  'lon': -94.4788154,
  'players': 4,
  'price': 50.0,
  'tee_time': 'Sat, 03 May 2025 16:5

## Weather Stuff

In [38]:
import requests
from datetime import datetime, timedelta

# Coordinates
lat = 39.2528261
lon = -94.4788154

# Get metadata from NOAA
point_url = f"https://api.weather.gov/points/{lat},{lon}"
headers = {
    "User-Agent": "my-weather-app (your@email.com)"
}

try:
  point_response = requests.get(point_url, headers=headers, timeout=10)
  point_response.raise_for_status()
  forecast_url = point_response.json()["properties"]["forecastHourly"]
except Exception as e:
  raise RuntimeError(f"Failed to get forecast URL from NOAA: {e}")

# Get forecast
try:
  forecast_response = requests.get(forecast_url, headers=headers, timeout=10)
  forecast_response.raise_for_status()
  forecast_periods = forecast_response.json()["properties"]["periods"]
except Exception as e:
  raise RuntimeError(f"Failed to fetch forecast: {e}")

# Get next Sunday
today = datetime.today()
days_ahead = (6 - today.weekday()) % 7  # 6 = Sunday
sunday = (today + timedelta(days=days_ahead)).strftime("%A")

# Find Sunday in forecast
sunday_forecast = next((p for p in forecast_periods if p["name"] == sunday), None)

if sunday_forecast:
  print(f"Forecast for Sunday at {lat}, {lon}:")
  print(f"{sunday_forecast['name']}: {sunday_forecast['detailedForecast']}")
else:
  print("Sunday forecast not found in NOAA response.")


Sunday forecast not found in NOAA response.


In [40]:
point_response.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/39.2528,-94.4788',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-94.4788, 39.2528]},
 'properties': {'@id': 'https://api.weather.gov/p

In [37]:
forecast_response.json()['properties']['periods']

[{'number': 1,
  'name': '',
  'startTime': '2025-05-02T07:00:00-05:00',
  'endTime': '2025-05-02T08:00:00-05:00',
  'isDaytime': True,
  'temperature': 54,
  'temperatureUnit': 'F',
  'temperatureTrend': '',
  'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 10},
  'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 9.444444444444445},
  'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 83},
  'windSpeed': '5 mph',
  'windDirection': 'SW',
  'icon': 'https://api.weather.gov/icons/land/day/bkn?size=small',
  'shortForecast': 'Partly Sunny',
  'detailedForecast': ''},
 {'number': 2,
  'name': '',
  'startTime': '2025-05-02T08:00:00-05:00',
  'endTime': '2025-05-02T09:00:00-05:00',
  'isDaytime': True,
  'temperature': 56,
  'temperatureUnit': 'F',
  'temperatureTrend': '',
  'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 16},
  'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 10},
  'relativeHumidity': {'unitCode': 'wmoUnit:percent